# CS4048 (Lec 08) Pandas III

Notebook adapted from Data 100, UC Berkeley

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading `babynames` Dataset for California

In [ ]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "/content/drive/MyDrive/Classroom/Data Science/namesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.tail(10)

,State,Sex,Year,Name,Count
420399,CA,M,2024,Zaxton,5
420400,CA,M,2024,Zaylin,5
420401,CA,M,2024,Zedekiah,5
420402,CA,M,2024,Zekai,5
420403,CA,M,2024,Zexi,5
420404,CA,M,2024,Ziad,5
420405,CA,M,2024,Ziah,5
420406,CA,M,2024,Zohaan,5
420407,CA,M,2024,Zubair,5
420408,CA,M,2024,Zuko,5


## Grouping

Group rows that share a common feature, then aggregate data across the group.

In this example, we count the total number of babies born each year in California.

In [ ]:
babynames.groupby("Year")

In [ ]:
# Grouping by "Year" and aggregating the "Count" column
# to get the total number of babies born each year.
babies_by_year = babynames.groupby("Year")[["Count"]].agg("sum")
babies_by_year

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26925
...,...
2020,363600
2021,363665
2022,362544


In [ ]:
# Plotting baby counts per year
# Don't worry about the syntax here. We will cover visualization later in the course.
fig = px.line(babies_by_year, y="Count")
fig.update_layout(font_size=18,
                  autosize=False,
                  width=700,
                  height=400)

### Exercise

Try to predict the results of the `groupby` operation shown. The answer is below the image.

<img src="images/groupby_mystery.png" alt="Image" width="600">

In [ ]:
df = pd.DataFrame({
  'col1' : ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'C', 'B'],
  'col2' : [3, 1, 4, 1, 5, 9, 2, 5, 6],
  'col3' : ['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']
})
df

,col1,col2,col3
0,A,3,ak
1,B,1,tx
2,C,4,fl
3,A,1,hi
4,B,5,mi
5,C,9,ak
6,A,2,ca
7,C,5,sd
8,B,6,nc


In [ ]:
# When we don't specify the columns, pandas will try to apply the aggregation to all columns. See next cell for proof!
df.groupby('col1').agg('max')

,col2,col3
col1,,
A,3,hi
B,6,tx
C,9,sd


In [ ]:
df.groupby('col1')[['col2', 'col3']].agg('max')

,col2,col3
col1,,
A,3,hi
B,6,tx
C,9,sd


### Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage in California. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [ ]:
f_babynames = babynames[babynames["Sex"]=="F"]
f_babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
246852,CA,F,2024,Zipporah,5
246853,CA,F,2024,Ziyah,5
246854,CA,F,2024,Zosia,5
246855,CA,F,2024,Zuleika,5


To build our intuition on how to answer our research question, let's visualize the prevalence of the name "Jennifer" over time.

In [ ]:
jenn_entries = f_babynames[f_babynames["Name"]=="Jennifer"]
jenn_entries

,State,Sex,Year,Name,Count
13610,CA,F,1934,Jennifer,5
16325,CA,F,1938,Jennifer,5
16993,CA,F,1939,Jennifer,6
17533,CA,F,1940,Jennifer,13
18210,CA,F,1941,Jennifer,24
...,...,...,...,...,...
228822,CA,F,2020,Jennifer,141
232602,CA,F,2021,Jennifer,91
236182,CA,F,2022,Jennifer,115
239981,CA,F,2023,Jennifer,88


In [ ]:
# We'll talk about how to generate plots in a later lecture
fig = px.line(jenn_entries, x="Year", y="Count")

fig.update_layout(font_size = 18,
                  autosize=False,
                  width=1000,
                  height=400)

We'll need a mathematical definition for the change in popularity of a name in California.

Define the metric "Ratio to Peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year.

A demo calculation for Jennifer:

In [ ]:
# Construct a Series containing our Jennifer count data
jenn_counts_ser = jenn_entries["Count"]
jenn_counts_ser

,Count
13610,5
16325,5
16993,6
17533,13
18210,24
...,...
228822,141
232602,91
236182,115
239981,88


In [ ]:
# In the year with the highest Jennifer count, 6065 Jennifers were born
max_jenn = np.max(jenn_counts_ser)
# jenn_counts_ser.max() -> 6065

max_jenn

6065

In [ ]:
# Remember that we sorted f_babynames by "Year".
# This means that grabbing the final entry gives us the most recent count of Jennifers: 80
# In 2024, the most recent year for which we have data, 80 Jennifers were born
latest_jenn = jenn_counts_ser.iloc[-1]
print(latest_jenn)

80


In [ ]:
# Compute the RTP
latest_jenn / max_jenn

np.float64(0.013190436933223413)

We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [ ]:
def ratio_to_peak(series):
    """
    Compute the RTP for a Series containing the counts per year for a single name (year column sorted ascendingly).
    """
    # return series.iloc[-1] / np.max(series)
    return series.iloc[-1] / series.max()

In [ ]:
# Then, find the RTP
ratio_to_peak(jenn_counts_ser)

np.float64(0.013190436933223413)

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in the lecture, `pandas` can't apply an aggregation function to non-numeric data (it doesn't make sense to divide "CA" by a number). We can select numerical columns of interest directly.

In [ ]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

,Year,Count
Name,,
Aadhini,1.0,1.000000
Aadhira,1.0,0.500000
Aadhya,1.0,0.820000
Aadya,1.0,1.000000
Aahana,1.0,0.307692
...,...,...
Zyanya,1.0,0.400000
Zyla,1.0,0.942857
Zylah,1.0,0.800000


This is the `pandas` equivalent of `.group` from [Data 8](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html). If we wanted to achieve this same result using the `datascience` library, we would write:

`f_babynames.group("Name", ratio_to_peak)`

### Exercise

Is there a row where `Year` is not equal to 1? Recall that `babynames` is sorted ascending by year.

In [ ]:
f_babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
246852,CA,F,2024,Zipporah,5
246853,CA,F,2024,Ziyah,5
246854,CA,F,2024,Zosia,5
246855,CA,F,2024,Zuleika,5


In [ ]:
# Unique values in the Year column
rtp_table["Year"].unique()

array([1.])

A hint: If we randomly shuffle the dataset, we see values of `Year` other than 1.
- The maximum year for each name is no longer guaranteed to be the last-appearing year for each name! So, the ratio is no longer 1.

In [ ]:
# .sample() method randomly selects a specified number or fraction of rows from the DataFrame.
# frac=1: Specifies that 100% of the rows (the entire DataFrame) should be selected.
# replace=False: Ensures that the sampling is done without replacement, meaning each row is used only once.

f_babynames.sample(frac=1, replace=False).groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)

,Year,Count
Name,,
Aadhini,1.000000,1.000000
Aadhira,0.997527,0.600000
Aadhya,1.000000,0.820000
Aadya,0.993083,0.241379
Aahana,0.999012,0.269231
...,...,...
Zyanya,0.990613,0.466667
Zyla,0.992095,0.142857
Zylah,0.998024,0.700000


In [ ]:
# Dropping the "Year" column
rtp_table = rtp_table.drop("Year", axis="columns")
rtp_table

,Count
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.820000
Aadya,1.000000
Aahana,0.307692
...,...
Zyanya,0.400000
Zyla,0.942857
Zylah,0.800000


In [ ]:
# Rename "Count" to "Count RTP" for clarity
rtp_table = rtp_table.rename(columns={"Count":"Count RTP"})
rtp_table

,Count RTP
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.820000
Aadya,1.000000
Aahana,0.307692
...,...
Zyanya,0.400000
Zyla,0.942857
Zylah,0.800000


In [ ]:
# What name has fallen the most in popularity?
rtp_table.sort_values("Count RTP", ascending=True)

,Count RTP
Name,
Debra,0.001512
Debbie,0.002817
Susan,0.003050
Tammy,0.003249
Cheryl,0.003819
...,...
Aaira,1.000000
Aahna,1.000000
Aadya,1.000000


We can visualize the decrease in the popularity of the name "Debra:"

In [ ]:
# The * in the parameter is used for multiple names
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)],
                  x="Year", y="Count", color="Name", # color = column name
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size=18,
                  autosize=False,
                  width=1000,
                  height=400)
    return fig

plot_name("Debra")

In [ ]:
# Find the 10 names that have decreased the most in popularity
top10 = rtp_table.sort_values("Count RTP").head(10).index
top10

Index(['Debra', 'Debbie', 'Susan', 'Tammy', 'Cheryl', 'Michele', 'Kathleen',
       'Terri', 'Shannon', 'Tina'],
      dtype='object', name='Name')

In [ ]:
plot_name(*top10)

For fun, try plotting your name or your friends' names.

### `groupby.size` and `groupby.count()`

In [ ]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.nan, 4],
                   'state':[np.nan, 'tx', 'fl', 'hi', np.nan, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [ ]:
df.groupby("letter").size()

,0
letter,
A,2
B,1
C,3


You might recall `value_counts()` function we talked about last week. What's the difference?

In [ ]:
df["letter"].value_counts()

,count
letter,
C,3
A,2
B,1


Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the values of the resulting `Series` in descending order.

`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
df.groupby("letter").count()

,num,state
letter,,
A,2,1
B,1,1
C,2,2


## Filtering by Group

Another common use for groups is to filter data:

Usage: `groupby(___).filter(func)`

`.filter()` applies `func` to each group's sub-DataFrame (`sf`).
- `func` must return a scalar `True` or `False` for each `sf`.
- If `func` returns `True` for a `sf`, then all rows belonging to the group are preserved.
- If `func` returns `False` for a `sf`, then all rows belonging to that group are filtered out.

For example, we can filter to the subframes of `df` with at least 2 rows:

In [ ]:
df.groupby("letter").filter(lambda sf: len(sf) >= 2)

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


### Exercise

Which of the following returns all rows of `babynames` with names that appeared for the first time after 2010?

In [ ]:
babynames.groupby("Name").filter(lambda sf: sf["Year"].min() > 2010)

,State,Sex,Year,Name,Count
195282,CA,F,2011,Mileidy,15
195296,CA,F,2011,Solara,15
195386,CA,F,2011,Yorley,14
195721,CA,F,2011,Mileydi,11
195848,CA,F,2011,Kensie,10
...,...,...,...,...,...
420402,CA,M,2024,Zekai,5
420403,CA,M,2024,Zexi,5
420405,CA,M,2024,Ziah,5
420406,CA,M,2024,Zohaan,5


In [ ]:
babynames.groupby("Name").filter(lambda sf: sf["Year"].max() > 2010)

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
420404,CA,M,2024,Ziad,5
420405,CA,M,2024,Ziah,5
420406,CA,M,2024,Zohaan,5
420407,CA,M,2024,Zubair,5


In [ ]:
babynames.groupby("Name").filter(lambda sf: sf["Year"] > 2010)

TypeError: filter function returned a Series, but expected a scalar bool

In [ ]:
babynames.groupby(["Name", "Year"]).filter(lambda sf: sf["Year"] > 2010)

TypeError: filter function returned a Series, but expected a scalar bool

---------------

In [ ]:
# Let's read the elections dataset
elections = pd.read_csv("/content/drive/MyDrive/Classroom/Data Science/elections.csv")
elections.sample(5)

,Year,Candidate,Party,Popular vote,Result,%
36,1880,James Garfield,Republican,4453337,win,48.369234
150,1996,Ross Perot,Reform,8085294,loss,8.408844
42,1888,Alson Streeter,Union Labor,146602,loss,1.288861
95,1940,Norman Thomas,Socialist,116599,loss,0.234237
92,1936,Norman Thomas,Socialist,187910,loss,0.412876


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [ ]:
elections.groupby("Year").filter(lambda sf: sf["%"].max() < 45).head(10)

,Year,Candidate,Party,Popular vote,Result,%
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408
24,1860,John Bell,Constitutional Union,590901,loss,12.639283
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
69,1912,William Taft,Republican,3486242,loss,23.218466
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
115,1968,George Wallace,American Independent,9901118,loss,13.571218


In [ ]:
# Why did we get a DataFrame instead of a Series?
# Notice that "%" is in its own sublist!
elections_max_percentage = elections.groupby("Year")[["%"]].agg("max")
elections_max_percentage

,%
Year,
1824,57.210122
1828,56.203927
1832,54.574789
1836,52.272472
1840,53.051213
1844,50.749477
1848,47.309296
1852,51.013168
1856,45.306080


In [ ]:
elections_max_percentage.sort_values(by="%").head()

,%
Year,
1860,39.699408
1912,41.933422
1992,43.118485
1968,43.565246
1856,45.306080


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [ ]:
elections.groupby("Party").agg("max").head(10)

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001
American Independent,1976,Lester Maddox,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Michael Peroutka,203091,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2024,Woodrow Wilson,81268924,win,61.344703


It's generally a good idea to be explicit about which columns to aggregate!

#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first few rows of the correct output should look like:

<img src="images/parties.png" alt="Image" width="600">

In [ ]:
elections_sorted_by_percent = elections.sort_values("%", ascending=False)
elections_sorted_by_percent.head(8)

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107
120,1972,Richard Nixon,Republican,47168710,win,60.907806
79,1920,Warren Harding,Republican,16144093,win,60.574501
133,1984,Ronald Reagan,Republican,54455472,win,59.023326
84,1928,Herbert Hoover,Republican,21427123,win,58.368524
86,1932,Franklin Roosevelt,Democratic,22821277,win,57.672125
109,1956,Dwight Eisenhower,Republican,35579180,win,57.650654


In [ ]:
elections_sorted_by_percent.groupby("Party").head(1)

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
120,1972,Richard Nixon,Republican,47168710,win,60.907806
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
27,1864,Abraham Lincoln,National Union,2211317,win,54.951512
11,1840,William Henry Harrison,Whig,1275583,win,53.051213
31,1872,Horace Greeley,Liberal Republican,2834761,loss,44.071406
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
22,1856,Millard Fillmore,American,873053,loss,21.554001


#### Alternative Solutions

You'll soon discover that with `Pandas` rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [ ]:
elections.groupby("Party")["%"].idxmax()

,%
Party,
American,22
American Independent,115
Anti-Masonic,6
Anti-Monopoly,38
Citizens,127
Communist,89
Constitution,164
Constitutional Union,24
Democratic,114


In [ ]:
# This is the computational part
best_per_party = elections.loc[elections.groupby("Party")["%"].idxmax()]

# This indexes by Party to match the formatting above
best_per_party.set_index('Party').sort_index().head()

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182


And this one doesn't even use `groupby`! This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.  Again, the 2nd line is purely formatting:

In [ ]:
best_per_party2 = elections.sort_values("%").drop_duplicates(["Party"], keep="last")
best_per_party2.set_index("Party").sort_index().head()  # Formatting

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [ ]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)

pandas.core.groupby.generic.DataFrameGroupBy

`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [ ]:
grouped_by_party.groups

{'American': [22, 126], 'American Independent': [115, 119, 124], 'Anti-Masonic': [6], 'Anti-Monopoly': [38], 'Citizens': [127], 'Communist': [89], 'Constitution': [160, 164, 172], 'Constitutional Union': [24], 'Democratic': [2, 4, 8, 10, 13, 14, 17, 20, 28, 29, 34, 37, 39, 45, 47, 52, 55, 57, 64, 70, 74, 77, 81, 83, 86, 91, 94, 97, 100, 105, 108, 111, 114, 116, 118, 123, 129, 134, 137, 140, 144, 151, 158, 162, 168, 176, 178, 183], 'Democratic-Republican': [0, 1], 'Dixiecrat': [103], 'Farmer–Labor': [78], 'Free Soil': [15, 18], 'Green': [149, 155, 156, 165, 170, 177, 181, 184], 'Greenback': [35], 'Independent': [121, 130, 143, 161, 167, 174, 185], 'Liberal Republican': [31], 'Libertarian': [125, 128, 132, 138, 139, 146, 153, 159, 163, 169, 175, 180], 'Libertarian Party': [186], 'National Democratic': [50], 'National Republican': [3, 5], 'National Union': [27], 'Natural Law': [148], 'New Alliance': [136], 'Northern Democratic': [26], 'Populist': [48, 61, 141], 'Progressive': [68, 82, 101

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [ ]:
grouped_by_party.get_group("Socialist")

,Year,Candidate,Party,Popular vote,Result,%
58,1904,Eugene V. Debs,Socialist,402810,loss,2.985897
62,1908,Eugene V. Debs,Socialist,420852,loss,2.850866
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
71,1916,Allan L. Benson,Socialist,590524,loss,3.194193
76,1920,Eugene V. Debs,Socialist,913693,loss,3.428282
85,1928,Norman Thomas,Socialist,267478,loss,0.728623
88,1932,Norman Thomas,Socialist,884885,loss,2.236211
92,1936,Norman Thomas,Socialist,187910,loss,0.412876
95,1940,Norman Thomas,Socialist,116599,loss,0.234237
102,1948,Norman Thomas,Socialist,139569,loss,0.286312


---

## Pivot Tables

### `Groupby` with multiple columns

We want to build a table showing the total number of babies born of each sex in each year. One way is to `groupby` using both columns of interest:

In [ ]:
# babynames.groupby(["Year", "Sex"])[["Count"]].sum()
babynames.groupby(["Year", "Sex"])[["Count"]].agg("sum")

Count
Year Sex        
1910 F      5950
     M      3213
1911 F      6602
     M      3381
1912 F      9804
...          ...
2022 M    188931
2023 F    164675
     M    179637
2024 F    164782
     M    179389

[230 rows x 1 columns]

### `pivot_table`

In [ ]:
babynames.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


In [ ]:
babynames.pivot_table(
    index="Year",
    columns="Sex",
    values="Count",
    aggfunc="sum")

Sex,F,M
Year,,
1910,5950,3213
1911,6602,3381
1912,9804,8142
1913,11860,10234
1914,13815,13110
...,...,...
2020,174112,189488
2021,174417,189248
2022,173613,188931


### `pivot_table` with Multiple values

In [ ]:
babynames.pivot_table(
    index="Year",
    columns="Sex",
    values=["Count", "Name"],
    aggfunc="max")

Count          Name         
Sex      F     M       F        M
Year                             
1910   295   237  Yvonne  William
1911   390   214   Zelma   Willis
1912   534   501  Yvonne  Woodrow
1913   584   614   Zelma   Yoshio
1914   773   769   Zelma   Yoshio
...    ...   ...     ...      ...
2020  2354  2633   Zyrah     Zyon
2021  2408  2622    Zyra    Zyrus
2022  2188  2617    Zyra     Zyon
2023  1975  2673    Zyra     Zyon
2024  1986  2716   Zyrah     Zyon

[115 rows x 4 columns]

---

## Join Tables

What if we want to know the popularity of presidential candidates' first names in California in 2022? What can we do?

In [ ]:
elections.head(10)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
9,1836,William Henry Harrison,Whig,550816,loss,37.721543


In [ ]:
babynames_2022 = babynames[babynames["Year"]==2022]
babynames_2022.head(10)

,State,Sex,Year,Name,Count
235881,CA,F,2022,Olivia,2188
235882,CA,F,2022,Emma,2087
235883,CA,F,2022,Camila,2050
235884,CA,F,2022,Mia,1890
235885,CA,F,2022,Sophia,1773
235886,CA,F,2022,Isabella,1741
235887,CA,F,2022,Luna,1524
235888,CA,F,2022,Sofia,1312
235889,CA,F,2022,Amelia,1295
235890,CA,F,2022,Emily,1111


In [ ]:
elections["First Name"] = elections["Candidate"].str.split(" ").str[0]
elections

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew
...,...,...,...,...,...,...,...
182,2024,Donald Trump,Republican,77303568,win,49.808629,Donald
183,2024,Kamala Harris,Democratic,75019230,loss,48.336772,Kamala
184,2024,Jill Stein,Green,861155,loss,0.554864,Jill
185,2024,Robert Kennedy,Independent,756383,loss,0.487357,Robert


The join function is called `merge` in pandas. `join` in pandas does something slightly different—we won't talk about it in this class.

In [ ]:
display(elections.head())
display(babynames_2022.head())

merged = pd.merge(left=elections, right=babynames_2022,
                  left_on="First Name", right_on="Name")
merged

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew


,State,Sex,Year,Name,Count
235881,CA,F,2022,Olivia,2188
235882,CA,F,2022,Emma,2087
235883,CA,F,2022,Camila,2050
235884,CA,F,2022,Mia,1890
235885,CA,F,2022,Sophia,1773


,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew,CA,M,2022,Andrew,747
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John,CA,M,2022,John,496
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew,CA,M,2022,Andrew,747
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John,CA,M,2022,John,496
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew,CA,M,2022,Andrew,747
...,...,...,...,...,...,...,...,...,...,...,...,...
153,2020,Howard Hawkins,Green,405035,loss,0.255731,Howard,CA,M,2022,Howard,18
154,2024,Donald Trump,Republican,77303568,win,49.808629,Donald,CA,M,2022,Donald,33
155,2024,Robert Kennedy,Independent,756383,loss,0.487357,Robert,CA,M,2022,Robert,409
156,2024,Chase Oliver,Libertarian Party,650130,loss,0.418895,Chase,CA,F,2022,Chase,6


In [ ]:
merged.sort_values("Count", ascending=False)

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
40,1888,Benjamin Harrison,Republican,5443633,win,47.858041,Benjamin,CA,M,2022,Benjamin,1533
36,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838,Benjamin,CA,M,2022,Benjamin,1533
42,1892,Benjamin Harrison,Republican,5176108,loss,42.984101,Benjamin,CA,M,2022,Benjamin,1533
33,1880,James B. Weaver,Greenback,308649,loss,3.352344,James,CA,M,2022,James,1097
21,1856,James Buchanan,Democratic,1835140,win,45.306080,James,CA,M,2022,James,1097
...,...,...,...,...,...,...,...,...,...,...,...,...
65,1912,Woodrow Wilson,Democratic,6296284,win,41.933422,Woodrow,CA,M,2022,Woodrow,6
156,2024,Chase Oliver,Libertarian Party,650130,loss,0.418895,Chase,CA,F,2022,Chase,6
78,1928,Herbert Hoover,Republican,21427123,win,58.368524,Herbert,CA,M,2022,Herbert,5
81,1932,Herbert Hoover,Republican,15761254,loss,39.830594,Herbert,CA,M,2022,Herbert,5
